In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
import io
import requests

In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PTCL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PTCL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\PTCL\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PTCL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import csv
import json
from elasticsearch import helpers, Elasticsearch

In [4]:
#### Reading 1000 articles from my public repository 

data= pd.read_csv("https://raw.githubusercontent.com/raheeltahir55/Datasets/main/metadata_100.csv")
data.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,04/07/2001,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,15/08/2000,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,25/08/2000,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,22/02/2001,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,11/05/2001,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [5]:
#### verifying elasticsearch

es = Elasticsearch()
res = requests.get('http://localhost:9200')
print(res.content)

b'{\n  "name" : "7UCQ8Z4",\n  "cluster_name" : "elasticsearch",\n  "cluster_uuid" : "xnqev71TR72TZ3F1uQEp2g",\n  "version" : {\n    "number" : "6.5.1",\n    "build_flavor" : "default",\n    "build_type" : "tar",\n    "build_hash" : "8c58350",\n    "build_date" : "2018-11-16T02:22:42.182257Z",\n    "build_snapshot" : false,\n    "lucene_version" : "7.5.0",\n    "minimum_wire_compatibility_version" : "5.6.0",\n    "minimum_index_compatibility_version" : "5.0.0"\n  },\n  "tagline" : "You Know, for Search"\n}\n'


In [6]:
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun

tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [ ]:
#### INDEXING
#### This block is indexing the collection.

import urllib.request
import codecs

url = "https://raw.githubusercontent.com/raheeltahir55/Datasets/main/metadata_100.csv"
ftpstream = urllib.request.urlopen(url)
csvfile = csv.reader(codecs.iterdecode(ftpstream, 'utf-8'))
index= 0
headers = []
for row in csvfile:
        try:
           if(index == 0):
                headers = row
           else:
                obj = {}
                for i, val in enumerate(row): 
                       #### checking for title, abstract and authors fields
                       if i==3 or i==8 or i==10:   
                          # Change all the text to lower case.
                          val= val.lower()
                          #### Tokenization and Normalization: creating tokens of 3 fields: title, abstract, authors
                          val= word_tokenize(val)
                          # Declaring Empty List to store the words that follow the rules for this step
                          Final_words = []
                          #### Stemming/Lemmatization: finding the base/dictionary form of a word
                          # Initializing WordNetLemmatizer()
                          word_Lemmatized = WordNetLemmatizer()
                          for word, tag in pos_tag(val):
                              #### checking for stop words and non alphabetical words  
                              if word not in stopwords.words('english') and word.isalpha():
                                 word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                                 Final_words.append(word_Final)
                          obj[headers[i]] = Final_words
                       else:
                          obj[headers[i]] = val 
                #### Indexing articles (row by row)
                es.index(index='covid_data_system1', doc_type='coviddata_19', body=obj)
                print(obj)
        except Exception as e:
          print('error: ' + str(e) + ' in' + str(index))
        index = index + 1  

c:\users\ptcl\appdata\local\programs\python\python39\lib\site-packages\elasticsearch\connection\base.py:193: ElasticsearchDeprecationWarning: Tue, 06 Apr 2021 11:51:31 GMT
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


{'cord_uid': 'ug7v899j', 'sha': 'd1aafb70c066a2068b02786f8929fd9c900897fb', 'source_x': 'PMC', 'title': ['clinical', 'feature', 'mycoplasma', 'pneumoniae', 'infection', 'king', 'abdulaziz', 'university', 'hospital', 'jeddah', 'saudi', 'arabia'], 'doi': '10.1186/1471-2334-1-6', 'pmcid': 'PMC35282', 'pubmed_id': '11472636', 'license': 'no-cc', 'abstract': ['objective', 'retrospective', 'chart', 'review', 'describe', 'epidemiology', 'clinical', 'feature', 'patient', 'mycoplasma', 'pneumoniae', 'infection', 'king', 'abdulaziz', 'university', 'hospital', 'jeddah', 'saudi', 'arabia', 'method', 'patient', 'positive', 'pneumoniae', 'culture', 'respiratory', 'specimen', 'january', 'december', 'identify', 'microbiology', 'record', 'chart', 'patient', 'review', 'result', 'patient', 'identify', 'required', 'admission', 'infection', 'infection', 'affect', 'age', 'group', 'common', 'infant', 'child', 'occur', 'common', 'fall', 'spring', 'patient', 'comorbidities', 'isolates', 'associate', 'pneumonia

In [ ]:
#### INDEXING for System 2 (without stopwords removal)
#### This block is indexing the collection.

import urllib.request
import codecs

url = "https://raw.githubusercontent.com/raheeltahir55/Datasets/main/metadata_100.csv"
ftpstream = urllib.request.urlopen(url)
csvfile = csv.reader(codecs.iterdecode(ftpstream, 'utf-8'))
index= 0
headers = []
for row in csvfile:
        try:
           if(index == 0):
                headers = row
           else:
                obj = {}
                for i, val in enumerate(row): 
                       #### checking for title, abstract and authors fields
                       if i==3 or i==8 or i==10:   
                          # Change all the text to lower case.
                          val= val.lower()
                          #### Tokenization and Normalization: creating tokens of 3 fields: title, abstract, authors
                          val= word_tokenize(val)
                          # Declaring Empty List to store the words that follow the rules for this step
                          Final_words = []
                          #### Stemming/Lemmatization: finding the base/dictionary form of a word
                          # Initializing WordNetLemmatizer()
                          word_Lemmatized = WordNetLemmatizer()
                          for word, tag in pos_tag(val):
                              #### checking for stop words and non alphabetical words  
                              if word.isalpha():
                                 word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                                 Final_words.append(word_Final)
                          obj[headers[i]] = Final_words
                       else:
                          obj[headers[i]] = val 
                #### Indexing articles (row by row)
                es.index(index='covid_data_system2', doc_type='coviddata_19', body=obj)
                print(obj)
        except Exception as e:
          print('error: ' + str(e) + ' in' + str(index))
        index = index + 1 

c:\users\ptcl\appdata\local\programs\python\python39\lib\site-packages\elasticsearch\connection\base.py:193: ElasticsearchDeprecationWarning: the default number of shards will change from [5] to [1] in 7.0.0; if you wish to continue using the default of [5] shards, you must manage this on the create index request or with an index template
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
c:\users\ptcl\appdata\local\programs\python\python39\lib\site-packages\elasticsearch\connection\base.py:193: ElasticsearchDeprecationWarning: Tue, 06 Apr 2021 10:37:25 GMT
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


{'cord_uid': 'ug7v899j', 'sha': 'd1aafb70c066a2068b02786f8929fd9c900897fb', 'source_x': 'PMC', 'title': ['clinical', 'feature', 'of', 'mycoplasma', 'pneumoniae', 'infection', 'at', 'king', 'abdulaziz', 'university', 'hospital', 'jeddah', 'saudi', 'arabia'], 'doi': '10.1186/1471-2334-1-6', 'pmcid': 'PMC35282', 'pubmed_id': '11472636', 'license': 'no-cc', 'abstract': ['objective', 'this', 'retrospective', 'chart', 'review', 'describe', 'the', 'epidemiology', 'and', 'clinical', 'feature', 'of', 'patient', 'with', 'mycoplasma', 'pneumoniae', 'infection', 'at', 'king', 'abdulaziz', 'university', 'hospital', 'jeddah', 'saudi', 'arabia', 'method', 'patient', 'with', 'positive', 'pneumoniae', 'culture', 'from', 'respiratory', 'specimen', 'from', 'january', 'through', 'december', 'be', 'identify', 'through', 'the', 'microbiology', 'record', 'chart', 'of', 'patient', 'be', 'review', 'result', 'patient', 'be', 'identify', 'of', 'whom', 'required', 'admission', 'most', 'infection', 'be', 'the', 'i

In [8]:
#### Pre-processing to calculate tf-tdf weights

doc_fields= ['title','abstract','authors']
for s in doc_fields:
    data[s]=data[s].astype(str)
f1=[]
f2=[] 
f3=[]
final_text= []
for f in doc_fields:
    # Change all the text to lower case.
    data[f] = [entry.lower() for entry in data[f]]

    # Tokenization : In this each entry in the corpus will be broken into set of words
    data[f]= [word_tokenize(entry) for entry in data[f]]

    for index, entry in enumerate(data[f]):
        # Declaring Empty List to store the words that follow the rules for this step
        Final_words = []
        # Initializing WordNetLemmatizer()
        word_Lemmatized = WordNetLemmatizer()
        # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
        for word, tag in pos_tag(entry):
            # Below condition is to check for Stop words and consider only alphabets
            if word not in stopwords.words('english') and word.isalpha():
                word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                Final_words.append(word_Final)
        # The final processed set of words for each iteration will be stored in 'text_final'
        if f== 'title':  
            f1.append(Final_words) 
        if f== 'abstract':
            f2.append(Final_words) 
        if f== 'authors':
            f3.append(Final_words) 
        data.loc[index, f] = str(Final_words)

In [9]:
data.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,"['clinical', 'feature', 'mycoplasma', 'pneumon...",10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,"['objective', 'retrospective', 'chart', 'revie...",04/07/2001,"['madani', 'tariq', 'aisha']",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,"['nitric', 'oxide', 'mediator', 'lung', 'disea...",10.1186/rr14,PMC59543,11667967,no-cc,"['inflammatory', 'disease', 'respiratory', 'tr...",15/08/2000,"['vliet', 'albert', 'van', 'der', 'eiserich', ...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,"['surfactant', 'pulmonary', 'host', 'defense']",10.1186/rr19,PMC59549,11667972,no-cc,"['surfactant', 'participate', 'innate', 'respo...",25/08/2000,"['crouch', 'erika', 'c']",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,"['role', 'lung', 'disease']",10.1186/rr44,PMC59574,11686871,no-cc,"['amino', 'acid', 'peptide', 'diverse', 'biolo...",22/02/2001,"['fagan', 'karen', 'mcmurtry', 'ivan', 'f', 'r...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,"['gene', 'expression', 'epithelial', 'cell', '...",10.1186/rr61,PMC59580,11686888,no-cc,"['respiratory', 'syncytial', 'virus', 'rsv', '...",11/05/2001,"['domachowske', 'joseph', 'b', 'bonville', 'cy...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [10]:
#### Merging the 3 processed fields into one so tf-idf vectorizer can fit on the entire processed data.

for i in range(data.shape[0]):
    data.loc[i, 'final_text']= str(f1[i]+f2[i]+f3[i])

data.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,final_text
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,"['clinical', 'feature', 'mycoplasma', 'pneumon...",10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,"['objective', 'retrospective', 'chart', 'revie...",04/07/2001,"['madani', 'tariq', 'aisha']",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN,"['clinical', 'feature', 'mycoplasma', 'pneumon..."
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,"['nitric', 'oxide', 'mediator', 'lung', 'disea...",10.1186/rr14,PMC59543,11667967,no-cc,"['inflammatory', 'disease', 'respiratory', 'tr...",15/08/2000,"['vliet', 'albert', 'van', 'der', 'eiserich', ...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,"['nitric', 'oxide', 'mediator', 'lung', 'disea..."
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,"['surfactant', 'pulmonary', 'host', 'defense']",10.1186/rr19,PMC59549,11667972,no-cc,"['surfactant', 'participate', 'innate', 'respo...",25/08/2000,"['crouch', 'erika', 'c']",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,"['surfactant', 'pulmonary', 'host', 'defense',..."
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,"['role', 'lung', 'disease']",10.1186/rr44,PMC59574,11686871,no-cc,"['amino', 'acid', 'peptide', 'diverse', 'biolo...",22/02/2001,"['fagan', 'karen', 'mcmurtry', 'ivan', 'f', 'r...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,"['role', 'lung', 'disease', 'amino', 'acid', '..."
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,"['gene', 'expression', 'epithelial', 'cell', '...",10.1186/rr61,PMC59580,11686888,no-cc,"['respiratory', 'syncytial', 'virus', 'rsv', '...",11/05/2001,"['domachowske', 'joseph', 'b', 'bonville', 'cy...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,"['gene', 'expression', 'epithelial', 'cell', '..."


In [11]:
#### Setting maxfeatures to 50000, meaning maximum of 50000 unique words can be used to calculate weights

Tfidf_vect = TfidfVectorizer(max_features=50000)
Tfidf_vect.fit(data['final_text'])

TfidfVectorizer(max_features=50000)

In [12]:
#### tf_idf is a sparse matrix with shape (1000, 17005)
#### 1000 articles with vector length 17005

tf_idf= Tfidf_vect.transform(data['final_text'])
tf_idf.shape

(1000, 17005)

In [13]:
print(tf_idf[0])

  (0, 16510)	0.07793981636470035
  (0, 16154)	0.058124902437322476
  (0, 15926)	0.064812003462083
  (0, 15866)	0.129624006924166
  (0, 15831)	0.08622254412428092
  (0, 15805)	0.05557280934817921
  (0, 15440)	0.05309163308595864
  (0, 15001)	0.08622254412428092
  (0, 14879)	0.04729008158859864
  (0, 14378)	0.07125271533993982
  (0, 14314)	0.056529275702502435
  (0, 14046)	0.04229277209062544
  (0, 14016)	0.05932084474757625
  (0, 13524)	0.16275491796248828
  (0, 13036)	0.07286637053327885
  (0, 13012)	0.05557280934817921
  (0, 12987)	0.04290026149479337
  (0, 12961)	0.07142840902386172
  (0, 12917)	0.06965708860511978
  (0, 12705)	0.06385553710775978
  (0, 12703)	0.06296998758035925
  (0, 12701)	0.05756901382422304
  (0, 12587)	0.03770248232084238
  (0, 12320)	0.05870791807742678
  (0, 12026)	0.031373891693888586
  :	:
  (0, 4254)	0.0397174310364376
  (0, 3903)	0.17796253424272873
  (0, 3780)	0.03624934394415477
  (0, 3610)	0.06585174158380361
  (0, 3552)	0.028768260508744592
  (0, 3385

In [14]:
#### Code onwards is extra code to compute cosine similarity of a query named input below with 
#### each of 1000 articles and showing the top 5 relevant articles

input= "covid vaccine"

In [15]:
#### Pre-processing the input user query

input = input.lower()

# Tokenization : In this each entry in the corpus will be broken into set of words
input = word_tokenize(input)

# Declaring Empty List to store the words that follow the rules for this step
Final_words = []
# Initializing WordNetLemmatizer()
word_Lemmatized = WordNetLemmatizer()
# pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
for word, tag in pos_tag(input):
    # Below condition is to check for Stop words and consider only alphabets
    if word not in stopwords.words('english') and word.isalpha():
        word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
        Final_words.append(word_Final)
input= Final_words
print(input)

['covid', 'vaccine']


In [16]:
inp= pd.Series(str(input))
print(inp)

0    ['covid', 'vaccine']
dtype: object


In [17]:
#### calculating tf-idf weights of query

tf_idf_1= Tfidf_vect.transform(inp)
tf_idf_1.shape

(1, 17005)

In [18]:
#### computing cosine similarity with each article

from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(tf_idf_1[0], tf_idf[0:1000]).flatten()
cosine_similarities

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.02622156, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.04850712, 0.05526145, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.04812025, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.16996208,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.03125247, 0.     

In [19]:
max= np.argmax(cosine_similarities)
print(max)

371


In [20]:
#### finding top 5 relevant articles

idx= (-cosine_similarities).argsort()[:5]
print(idx)

[371 368 106 980 154]


In [21]:
#### printing top 5 relevant documents

for i in idx:
    print(data.iloc[i, :])
    print('\n')

cord_uid                                                     cwzpb8fu
sha                          56e040a2052581a7133f24e50dc7261b579fe22f
source_x                                                          PMC
title               ['confront', 'potential', 'influenza', 'pandem...
doi                                            10.3201/eid1310.061262
pmcid                                                      PMC2851514
pubmed_id                                                    18258000
license                                                         no-cc
abstract            ['influenza', 'virus', 'strong', 'candidate', ...
publish_time                                               31/10/2007
authors             ['haque', 'azizul', 'hober', 'didier', 'kasper...
journal                                              Emerg Infect Dis
mag_id                                                            NaN
who_covidence_id                                                  NaN
arxiv_id            